In [2]:
from datasets import load_dataset
raw_dataset = load_dataset(path="rugds/ditec-wdn", data_files="EXN_2GB_24H/junction_elevation*")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

EXN_2GB_24H/junction_elevation-0-static_(…):   0%|          | 0.00/18.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [1]:
# %% [markdown]
# # 💻 Google Colab: Conformal Prediction-Driven Adaptive Sampling for WDN DTs
#
# **Based on the paper: "Conformal Prediction-Driven Adaptive Sampling for Digital Twins of Water Distribution Networks."**
#
# This notebook implements the core logic: LSTM forecasting, Conformal Prediction calibration, and the adaptive sensor selection strategy.
#
# **IMPORTANT NOTE:** This code uses synthetic data to simulate the DiTEC-WDN structure for rapid execution. For *exact replication* of the paper's hydraulic results ($\text{RMSE}_p$ and $V_{\text{rate}}$), you must replace the pressure simulation proxy (Section 4) with an actual EPANET-based solver (e.g., WNTR or EPYNET) and load the full DiTEC-WDN dataset and corresponding network configuration files.
#
# **Required Library Installation:**

# %%
# Install necessary libraries (assuming Colab environment has basic packages like numpy/pandas/torch)
!pip install tqdm scikit-learn

# %% [markdown]
# ## 1. Setup and Data Loading

# %%
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import math
import os

# Set the seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Define device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- CONFIGURATION PARAMETERS ---
NETWORK_NAME = 'Hanoi' # Placeholder for the network under test
N_NODES = 31 # Corresponding to Hanoi network (31 junctions)
TIME_STEPS = 1000 # Reduced from 8760 for fast execution
N_SCENARIOS = 100 # Reduced from 1000 for fast execution

# Split ratios (60/20/20 for Train/Cal/Test, per Section 4.1)
TRAIN_SCENARIOS = int(N_SCENARIOS * 0.6)
CAL_SCENARIOS = int(N_SCENARIOS * 0.2)
TEST_SCENARIOS = N_SCENARIOS - TRAIN_SCENARIOS - CAL_SCENARIOS

LOOKBACK_WINDOW = 24 # w=24 hours (Section 3.2)
LSTM_UNITS = 64 # d=64 (Section 3.2)
ALPHA = 0.1 # 1-alpha = 90% target coverage (Section 3.3)
BUDGET_PERCENT = 0.4 # B=40% coverage for the main comparison
BUDGET_B = math.ceil(N_NODES * BUDGET_PERCENT)

print(f"Network: {NETWORK_NAME}, Total Nodes: {N_NODES}, Budget B (Sensors): {BUDGET_B}")

# %% [markdown]
# ### 1.1 Synthetic DiTEC-WDN Data Generation (Proxy)
# Simulating demand ($q$) and pressure ($p$) data structure.

# %%
# --- Synthetic DiTEC-WDN Data Generation (Proxy for actual data) ---
# Shape: (N_SCENARIOS, TIME_STEPS, N_NODES)
print("Generating synthetic data to simulate DiTEC-WDN...")

# Base daily pattern (smooth sine wave) + noise
base_demand = np.sin(np.linspace(0, 4*np.pi, TIME_STEPS))[:, None] * 10 + 20
base_pressure = 40 - np.sin(np.linspace(0, 4*np.pi, TIME_STEPS))[:, None] * 5

# Generate demand (q) and pressure (p) for all nodes and scenarios
q_data = np.zeros((N_SCENARIOS, TIME_STEPS, N_NODES))
p_data = np.zeros((N_SCENARIOS, TIME_STEPS, N_NODES))

for s in range(N_SCENARIOS):
    # Add random node-specific bias and noise to simulate heterogeneity
    node_bias_q = np.random.uniform(0.5, 1.5, N_NODES)
    node_bias_p = np.random.uniform(0.8, 1.2, N_NODES)

    # Time-series noise
    noise_q = np.random.normal(0, 0.5, (TIME_STEPS, N_NODES))
    noise_p = np.random.normal(0, 0.1, (TIME_STEPS, N_NODES))

    q_data[s] = (base_demand * node_bias_q) + noise_q
    q_data[s] = np.maximum(0, q_data[s]) # Demands must be non-negative

    p_data[s] = (base_pressure * node_bias_p) + noise_p

# Split into sets
q_train = q_data[:TRAIN_SCENARIOS]
q_cal = q_data[TRAIN_SCENARIOS:TRAIN_SCENARIOS + CAL_SCENARIOS]
q_test = q_data[TRAIN_SCENARIOS + CAL_SCENARIOS:]

p_test = p_data[TRAIN_SCENARIOS + CAL_SCENARIOS:] # p_test used only for proxy in evaluation

print(f"Data shapes: q_train {q_train.shape}, q_cal {q_cal.shape}, q_test {q_test.shape}")


# %% [markdown]
# ### 1.2 PyTorch Dataset Utility

# %%
class WDNTimeSeriesDataset(Dataset):
    """
    Dataset to prepare lookback window sequences for LSTM training.
    Handles multiple scenarios and time steps.
    """
    def __init__(self, data, window_size):
        self.data = data # Shape: (N_SCENARIOS, TIME_STEPS, N_NODES)
        self.window_size = window_size
        self.N_NODES = data.shape[2]
        self.N_SCENARIOS = data.shape[0]
        self.TIME_STEPS = data.shape[1]

        # Flatten and index all possible (scenario, time_step) pairs
        self.indices = []
        for s in range(self.N_SCENARIOS):
            # Start from window_size to ensure full lookback history
            for t in range(self.window_size, self.TIME_STEPS):
                self.indices.append((s, t))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        s, t = self.indices[idx]

        # Input: q_t-w:t-1 (w=window_size)
        x = self.data[s, t - self.window_size:t, :]

        # Target: q_t
        y = self.data[s, t, :]

        # Returns (seq_len, num_features) = (w, N_NODES)
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Prepare the training DataLoader (used for all nodes)
train_dataset = WDNTimeSeriesDataset(q_train, LOOKBACK_WINDOW)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


# %% [markdown]
# ## 2. LSTM Model Definition and Training

# %%
class NodeLSTM(nn.Module):
    """
    Independent LSTM model for demand forecasting for a single node.
    Architecture: 2 LSTM layers, d=64, lookback w=24 (Section 3.2).
    """
    def __init__(self, input_size, hidden_size, num_layers=2, dropout=0.2):
        super(NodeLSTM, self).__init__()
        # input_size is 1 for a single node's demand
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.linear = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU() # For non-negative demands

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size=1)
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :] # shape: (batch_size, hidden_size)
        output = self.linear(last_output) # shape: (batch_size, 1)
        return self.relu(output)


# %%
def train_lstm_for_node(node_idx, train_loader, epochs=100, lr=1e-3, patience=10):
    """
    Trains an independent LSTM model for a specific node's demand.
    """
    model = NodeLSTM(input_size=1, hidden_size=LSTM_UNITS).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5) # L2 regularization (Section 3.2)
    criterion = nn.MSELoss()

    best_loss = float('inf')
    patience_counter = 0

    for epoch in tqdm(range(epochs), desc=f"Node {node_idx} Training", leave=False):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            # Select only the data for the current node
            x_node = x_batch[:, :, node_idx:node_idx+1].to(DEVICE) # (batch, seq_len, 1)
            y_node = y_batch[:, node_idx:node_idx+1].to(DEVICE)   # (batch, 1)

            optimizer.zero_grad()
            y_pred = model(x_node)
            loss = criterion(y_pred, y_node)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x_batch.size(0)

        avg_loss = total_loss / len(train_dataset)

        # Early stopping logic (simplified)
        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    return model.cpu() # Return model to CPU


# %% [markdown]
# ### 2.1 Training All Node Models
# **NOTE:** The number of epochs is intentionally reduced (to 10) for quick execution in Colab. Increase `epochs` (e.g., to 100) for better accuracy, as used in the paper.

# %%
node_models = {}
print(f"Starting training for {N_NODES} independent LSTM models...")
for i in tqdm(range(N_NODES), desc="Total Model Training"):
    # Using 10 epochs for demonstration speed.
    node_models[i] = train_lstm_for_node(i, train_loader, epochs=10, patience=5)
print("Training complete.")


# %% [markdown]
# ## 3. Conformal Prediction Calibration

# %%
def get_autoregressive_residuals(model, q_data, lookback_window, node_idx):
    """
    Generates non-conformity scores (|q_true - q_hat|) using the calibration data.

    A SIMPLIFIED approach is used here, assuming true history is available for calibration
    to avoid full recursive simulation complexity during this phase.
    The resulting residual set $\{s_j^{(i)}\}$ is used to find the quantile $\hat{Q}^{(i)}_{1-\alpha}$.
    """
    all_residuals = []

    # Flatten the data to simulate a single long sequence of time steps
    q_data_flat = q_data.reshape(-1, N_NODES)
    N_FLAT_STEPS = q_data_flat.shape[0]

    for t in range(lookback_window, N_FLAT_STEPS):
        # Current time step is t.

        # 1. Get history: q_t-w:t-1 (using true values for calibration history)
        history = q_data_flat[t - lookback_window:t, node_idx]
        x = torch.tensor(history, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(DEVICE) # (1, w, 1)

        # 2. Predict demand (q-hat)
        model.eval()
        with torch.no_grad():
            q_hat = model(x).item()

        # 3. Get true demand (q)
        q_true = q_data_flat[t, node_idx]

        # 4. Residual (non-conformity score): s = |q - q_hat|
        residual = abs(q_true - q_hat)
        all_residuals.append(residual)

    return all_residuals

# %%
# --- Conformal Prediction Calibration ---

node_quantiles = {}
print(f"Starting CP calibration for {N_NODES} nodes...")

for node_idx in tqdm(range(N_NODES), desc="CP Calibration"):
    model = node_models[node_idx].to(DEVICE)

    # Get residuals (non-conformity scores)
    residuals = get_autoregressive_residuals(
        model,
        q_cal,
        LOOKBACK_WINDOW,
        node_idx
    )

    n_cal = len(residuals)

    # Calculate the quantile index (Section 3.3, Equation 7)
    quantile_rank = math.ceil((1 - ALPHA) * (n_cal + 1))

    # Sort residuals to find the quantile
    sorted_residuals = np.sort(residuals)

    # Get the quantile (non-conformity score Q_1-alpha)
    # This is the radius of the prediction interval.
    Q_1_alpha = sorted_residuals[quantile_rank - 1]

    node_quantiles[node_idx] = Q_1_alpha

print("CP Calibration complete.")


# %% [markdown]
# ## 4. Adaptive Sampling and Simulation

# %%
def adaptive_sampling_simulation(q_true, p_true, models, quantiles, B, lookback_window):
    """
    Implements Algorithm 1: CP-Guided Adaptive Sampling on the test set.
    """
    N_SCEN, T, N = q_true.shape

    # Output storage
    q_tilde = np.zeros_like(q_true) # Hybrid demand (measured or predicted)
    q_hat = np.zeros_like(q_true) # Predicted demand (for all nodes)
    U_scores = np.zeros_like(q_tilde) # Uncertainty scores (conformal interval width)

    # --- Full Simulation Loop (across all test scenarios) ---
    for s in tqdm(range(N_SCEN), desc="Test Scenarios Simulation"):
        # We process one scenario at a time, autoregressively
        q_scenario = q_true[s] # (T, N)
        p_scenario = p_true[s] # (T, N)

        # Initialize history for autoregressive prediction. Uses true values initially.
        q_history_buffer = q_scenario.copy() # (T, N) - We update this in place.

        for t in range(lookback_window, T):
            U_t = np.zeros(N)
            q_hat_t = np.zeros(N)

            # --- 1. Predict All Nodes & Compute Uncertainty (Algorithm 1, Lines 2-5) ---
            for i in range(N):
                model = models[i].to(DEVICE)

                # Get history: q_t-w:t-1 from the *buffer* (autoregressive)
                history = q_history_buffer[t-lookback_window:t, i]
                x = torch.tensor(history, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(DEVICE)

                model.eval()
                with torch.no_grad():
                    q_hat_i = model(x).item()

                # U_t(i) = 2 * Q_1-alpha (Section 3.3, Equation 8)
                U_i = 2 * quantiles[i]

                q_hat_t[i] = q_hat_i
                U_t[i] = U_i

            q_hat[s, t] = q_hat_t
            U_scores[s, t] = U_t

            # --- 2. Select Top-B Uncertain Nodes (Algorithm 1, Line 6) ---
            # Greedy selection: argmax sum(U_t(i)) (Section 3.1, Equation 4)
            selected_indices = np.argsort(U_t)[-B:] # Sort ascending, take last B

            # --- 3. Construct Hybrid State (Algorithm 1, Lines 7-9) ---
            q_tilde_t = np.zeros(N)
            for i in range(N):
                if i in selected_indices:
                    # Measured: q_tilde = q_true
                    q_tilde_t[i] = q_scenario[t, i]
                else:
                    # Predicted: q_tilde = q_hat
                    q_tilde_t[i] = q_hat_t[i]

            q_tilde[s, t] = q_tilde_t

            # Update history buffer with the current hybrid state (Crucial for autoregression)
            q_history_buffer[t] = q_tilde_t

            # --- 4. Hydraulic Simulation (Algorithm 1, Line 10) ---
            # [p_tilde, h_tilde] = EPANET(q_tilde, G)
            # SIMULATION PROXY: We use the ground truth pressure p_true as a proxy for the resulting p_tilde.
            # Replace this with a WNTR/EPYNET call for full hydraulic replication.
            # p_tilde_t = p_scenario[t]

    return q_tilde, q_hat, U_scores

# %%
# --- Execute Adaptive Sampling Simulation ---

q_tilde_adaptive, q_hat_adaptive, U_scores_adaptive = adaptive_sampling_simulation(
    q_test,
    p_test,
    node_models,
    node_quantiles,
    BUDGET_B,
    LOOKBACK_WINDOW
)
print("Adaptive sampling simulation complete.")

# --- Prepare data slices for Evaluation (skipping the lookback window) ---
q_test_slice = q_test[:, LOOKBACK_WINDOW:, :]
q_tilde_adaptive_slice = q_tilde_adaptive[:, LOOKBACK_WINDOW:, :]
q_hat_slice = q_hat_adaptive[:, LOOKBACK_WINDOW:, :]

# --- Baseline: Uniform Random Sampling ---
# We reuse the same trained LSTM predictions (q_hat_slice) for a fair comparison.
T, N = q_test_slice.shape[1], q_test_slice.shape[2]
q_tilde_uniform = np.zeros_like(q_test_slice)

for s in range(q_test_slice.shape[0]):
    for t in range(T):
        # Uniformly sample B nodes
        sampled_indices = np.random.choice(N, BUDGET_B, replace=False)
        for i in range(N):
            if i in sampled_indices:
                q_tilde_uniform[s, t, i] = q_test_slice[s, t, i] # Measured (True)
            else:
                q_tilde_uniform[s, t, i] = q_hat_slice[s, t, i] # Predicted (LSTM)


# %% [markdown]
# ## 5. Evaluation

# %%
# Flatten data for metric calculation
q_true_flat = q_test_slice.flatten()
q_tilde_adaptive_flat = q_tilde_adaptive_slice.flatten()
q_tilde_uniform_flat = q_tilde_uniform.flatten()

# 1. Demand RMSE (Eq. 9)
def calculate_rmse(q_true, q_tilde):
    return np.sqrt(np.mean((q_true - q_tilde)**2))

rmse_q_adaptive = calculate_rmse(q_true_flat, q_tilde_adaptive_flat)
rmse_q_uniform = calculate_rmse(q_true_flat, q_tilde_uniform_flat)

# 2. Coverage (Empirical) (Eq. 11)
def calculate_coverage(q_true_slice, q_hat_slice, quantiles, N_nodes):
    """
    Calculates the empirical coverage based on CP intervals:
    Checks if q_true is in [q_hat - Q_1-alpha, q_hat + Q_1-alpha]
    """
    coverage_count = 0
    total_samples = q_true_slice.size

    q_true_2d = q_true_slice.reshape(-1, N_nodes)
    q_hat_2d = q_hat_slice.reshape(-1, N_nodes)

    for i in range(N_nodes):
        Q_i = quantiles[i]
        q_true_i = q_true_2d[:, i]
        q_hat_i = q_hat_2d[:, i]

        lower_bound = q_hat_i - Q_i
        upper_bound = q_hat_i + Q_i

        is_covered = (q_true_i >= lower_bound) & (q_true_i <= upper_bound)
        coverage_count += np.sum(is_covered)

    empirical_coverage = coverage_count / total_samples
    return empirical_coverage * 100

coverage_adaptive = calculate_coverage(q_test_slice, q_hat_slice, node_quantiles, N_NODES)
target_coverage = (1 - ALPHA) * 100


# %% [markdown]
# ### 5.1 Results Summary

# %%
print("## 📊 Evaluation Results (Synthetic Data Proxy)")
print(f"Network: {NETWORK_NAME}, Sampling Budget: {BUDGET_PERCENT*100:.0f}% (B={BUDGET_B} sensors)")

print("\n--- Demand Estimation RMSE (L/s) ---")
print(f"Ours (Adaptive): {rmse_q_adaptive:.4f}")
print(f"Uniform Random: {rmse_q_uniform:.4f}")
improvement = (rmse_q_uniform - rmse_q_adaptive) / rmse_q_uniform * 100
print(f"Improvement over Uniform: {improvement:.2f}%")

print("\n--- Conformal Prediction Coverage (%) ---")
print(f"Target Coverage (1-alpha): {target_coverage:.1f}%")
print(f"Ours (Adaptive) Empirical Coverage: {coverage_adaptive:.2f}%")

<>:240: SyntaxWarning: invalid escape sequence '\{'
<>:240: SyntaxWarning: invalid escape sequence '\{'
/tmp/ipython-input-168836746.py:240: SyntaxWarning: invalid escape sequence '\{'
  The resulting residual set $\{s_j^{(i)}\}$ is used to find the quantile $\hat{Q}^{(i)}_{1-\alpha}$.


Using device: cpu
Network: Hanoi, Total Nodes: 31, Budget B (Sensors): 13
Generating synthetic data to simulate DiTEC-WDN...
Data shapes: q_train (60, 1000, 31), q_cal (20, 1000, 31), q_test (20, 1000, 31)
Starting training for 31 independent LSTM models...


Total Model Training:   0%|          | 0/31 [00:00<?, ?it/s]

Node 0 Training:   0%|          | 0/10 [00:00<?, ?it/s]

Node 1 Training:   0%|          | 0/10 [00:00<?, ?it/s]

Node 2 Training:   0%|          | 0/10 [00:00<?, ?it/s]

Node 3 Training:   0%|          | 0/10 [00:00<?, ?it/s]

Node 4 Training:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

# 💻 Conformal Prediction-Driven Adaptive Sampling for WDN DTs

**This Google Colab notebook implements the core methodology of the paper: LSTM Forecasting, Conformal Prediction (CP) for uncertainty quantification, and the Adaptive Sampling Policy.**

---

## 1. Setup and Data Loading

**Goal:** Install dependencies, set configuration parameters (nodes, budgets, $\alpha$), and generate synthetic data to simulate the DiTEC-WDN structure.

**CRITICAL NOTE:** For full hydraulic replication (Pressure $\text{RMSE}$, $V_{\text{rate}}$), this synthetic data section must be replaced by the actual loading of DiTEC-WDN Parquet files and EPANET integration.

---

In [ ]:
# Install necessary libraries
!pip install tqdm scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import math
import os

# Set the seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Define device (Use GPU if available for faster training)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

### 1.2 Configuration Parameters (Based on Hanoi Network and Paper's Setup)

In [ ]:
# --- CONFIGURATION PARAMETERS ---
NETWORK_NAME = 'Hanoi'
N_NODES = 31 # Corresponding to Hanoi network
TIME_STEPS = 1000 # Reduced from 8760 for quick execution
N_SCENARIOS = 100 # Reduced from 1000 for quick execution

# Split ratios (Train/Cal/Test: 60/20/20)
TRAIN_SCENARIOS = int(N_SCENARIOS * 0.6)
CAL_SCENARIOS = int(N_SCENARIOS * 0.2)
TEST_SCENARIOS = N_SCENARIOS - TRAIN_SCENARIOS - CAL_SCENARIOS

LOOKBACK_WINDOW = 24 # w=24 hours (Section 3.2)
LSTM_UNITS = 64 # d=64 (Section 3.2)
ALPHA = 0.1 # Conformal Significance Level (1-alpha = 90% coverage)
BUDGET_PERCENT = 0.4 # Sampling Budget (40% coverage)
BUDGET_B = math.ceil(N_NODES * BUDGET_PERCENT)

print(f"Network: {NETWORK_NAME}, Total Nodes: {N_NODES}, Sampling Budget B: {BUDGET_B}")

### 1.3 Synthetic DiTEC-WDN Data Generation

Generating synthetic demand ($q$) and pressure ($p$) data with heterogeneity to simulate the DiTEC-WDN structure.

In [ ]:
# --- Synthetic DiTEC-WDN Data Generation (Proxy) ---
print("Generating synthetic demand (q) and pressure (p) data...")

# Base daily pattern (smooth sine wave) + noise
base_demand = np.sin(np.linspace(0, 4*np.pi, TIME_STEPS))[:, None] * 10 + 20
base_pressure = 40 - np.sin(np.linspace(0, 4*np.pi, TIME_STEPS))[:, None] * 5

q_data = np.zeros((N_SCENARIOS, TIME_STEPS, N_NODES))
p_data = np.zeros((N_SCENARIOS, TIME_STEPS, N_NODES))

for s in range(N_SCENARIOS):
    node_bias_q = np.random.uniform(0.5, 1.5, N_NODES)
    node_bias_p = np.random.uniform(0.8, 1.2, N_NODES)
    noise_q = np.random.normal(0, 0.5, (TIME_STEPS, N_NODES))

    q_data[s] = (base_demand * node_bias_q) + noise_q
    q_data[s] = np.maximum(0, q_data[s])
    p_data[s] = (base_pressure * node_bias_p) + np.random.normal(0, 0.1, (TIME_STEPS, N_NODES))

# Split into sets
q_train = q_data[:TRAIN_SCENARIOS]
q_cal = q_data[TRAEN_SCENARIOS:TRAIN_SCENARIOS + CAL_SCENARIOS]
q_test = q_data[TRAIN_SCENARIOS + CAL_SCENARIOS:]
p_test = p_data[TRAIN_SCENARIOS + CAL_SCENARIOS:]

print(f"Data shapes: q_train {q_train.shape}, q_cal {q_cal.shape}, q_test {q_test.shape}")

### 1.4 PyTorch Dataset Utility

Defining the PyTorch Dataset class to manage time-series sequences with the specified lookback window ($w=24$).

In [ ]:
class WDNTimeSeriesDataset(Dataset):
    """
    Dataset to prepare lookback window sequences for LSTM training.
    """
    def __init__(self, data, window_size):
        self.data = data
        self.window_size = window_size

        self.indices = []
        for s in range(self.data.shape[0]):
            for t in range(self.data.shape[1]):
                 # We must have enough history available for the lookback window
                if t >= self.window_size:
                    self.indices.append((s, t))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        s, t = self.indices[idx]
        x = self.data[s, t - self.window_size:t, :] # History (w, N_NODES)
        y = self.data[s, t, :] # Target (N_NODES)

        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Prepare the training DataLoader
train_dataset = WDNTimeSeriesDataset(q_train, LOOKBACK_WINDOW)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

---

## 2. LSTM Model Definition and Training

**Goal:** Define the independent LSTM architecture and train one model for each node ($N=31$) using the training dataset.

---

In [ ]:
class NodeLSTM(nn.Module):
    """
    Independent LSTM model (2 layers, 64 units) for single node demand forecasting.
    """
    def __init__(self, input_size, hidden_size, num_layers=2, dropout=0.2):
        super(NodeLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout
        )
        self.linear = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU() # Non-negative demands

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        output = self.linear(last_output)
        return self.relu(output)

def train_lstm_for_node(node_idx, train_loader, epochs=100, lr=1e-3, patience=10):
    """Trains an independent LSTM model for a specific node."""
    model = NodeLSTM(input_size=1, hidden_size=LSTM_UNITS).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    criterion = nn.MSELoss()

    best_loss = float('inf')
    patience_counter = 0

    for epoch in tqdm(range(epochs), desc=f"Node {node_idx} Training", leave=False):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            x_node = x_batch[:, :, node_idx:node_idx+1].to(DEVICE)
            y_node = y_batch[:, node_idx:node_idx+1].to(DEVICE)

            optimizer.zero_grad()
            loss = criterion(model(x_node), y_node)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * x_batch.size(0)

        avg_loss = total_loss / len(train_dataset)

        if avg_loss < best_loss:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    return model.cpu()

### 2.2 Training Execution

Training $N=31$ independent models. (Using 10 epochs for quick execution; increase for production quality).

In [ ]:
node_models = {}
print(f"Starting training for {N_NODES} independent LSTM models...")
for i in tqdm(range(N_NODES), desc="Total Model Training"):
    node_models[i] = train_lstm_for_node(i, train_loader, epochs=10, patience=5)
print("Training complete. Models are ready for calibration.")

---

## 3. Conformal Prediction Calibration

**Goal:** Determine the non-conformity score quantile ($\hat{Q}^{(i)}_{1-\alpha}$) for each node $i$ using the calibration dataset ($q_{\text{cal}}$). This quantile defines the uncertainty radius ($U = 2\hat{Q}^{(i)}_{1-\alpha}$).

---

In [ ]:
def get_autoregressive_residuals(model, q_data, lookback_window, node_idx):
    """Generates non-conformity scores (|q_true - q_hat|) on the calibration set."""
    all_residuals = []
    q_data_flat = q_data.reshape(-1, N_NODES)
    N_FLAT_STEPS = q_data_flat.shape[0]

    for t in range(lookback_window, N_FLAT_STEPS):
        # Use true history for calibration set (simplified approach)
        history = q_data_flat[t - lookback_window:t, node_idx]
        x = torch.tensor(history, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(DEVICE)

        model.eval()
        with torch.no_grad():
            q_hat = model(x).item()

        q_true = q_data_flat[t, node_idx]
        residual = abs(q_true - q_hat)
        all_residuals.append(residual)

    return all_residuals

# --- Calibration Execution ---
node_quantiles = {}
print("Starting CP calibration...")

for node_idx in tqdm(range(N_NODES), desc="CP Calibration"):
    model = node_models[node_idx].to(DEVICE)
    residuals = get_autoregressive_residuals(model, q_cal, LOOKBACK_WINDOW, node_idx)

    n_cal = len(residuals)
    # Calculate the quantile rank (Equation 7)
    quantile_rank = math.ceil((1 - ALPHA) * (n_cal + 1))

    sorted_residuals = np.sort(residuals)

    # Get the quantile (Q_1-alpha, the prediction interval radius)
    Q_1_alpha = sorted_residuals[quantile_rank - 1]

    node_quantiles[node_idx] = Q_1_alpha

print("CP Calibration complete. Quantiles stored for adaptive sampling.")

---

## 4. Adaptive Sampling and Simulation

**Goal:** Implement Algorithm 1 (The core adaptive loop). At each step, predict all demands, select the $B$ most uncertain nodes based on CP interval width ($U$), and form the hybrid state ($\tilde{q}$).

---

In [ ]:
def adaptive_sampling_simulation(q_true, p_true, models, quantiles, B, lookback_window):
    """Implements Algorithm 1: CP-Guided Adaptive Sampling."""
    N_SCEN, T, N = q_true.shape

    q_tilde = np.zeros_like(q_true) # Hybrid demand
    q_hat = np.zeros_like(q_true) # Predicted demand
    U_scores = np.zeros_like(q_tilde) # Uncertainty scores

    for s in tqdm(range(N_SCEN), desc="Test Scenarios Simulation"):
        q_scenario = q_true[s]
        p_scenario = p_true[s]
        q_history_buffer = q_scenario.copy()

        for t in range(lookback_window, T):
            U_t = np.zeros(N)
            q_hat_t = np.zeros(N)

            # 1. Predict All Nodes & Compute Uncertainty
            for i in range(N):
                model = models[i].to(DEVICE)
                history = q_history_buffer[t-lookback_window:t, i]
                x = torch.tensor(history, dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(DEVICE)

                model.eval()
                with torch.no_grad():
                    q_hat_i = model(x).item()

                U_i = 2 * quantiles[i] # Uncertainty Score (Interval Width)

                q_hat_t[i] = q_hat_i
                U_t[i] = U_i

            q_hat[s, t] = q_hat_t
            U_scores[s, t] = U_t

            # 2. Select Top-B Uncertain Nodes (Greedy Policy)
            selected_indices = np.argsort(U_t)[-B:]

            # 3. Construct Hybrid State
            q_tilde_t = np.zeros(N)
            for i in range(N):
                if i in selected_indices:
                    q_tilde_t[i] = q_scenario[t, i] # Measured (True)
                else:
                    q_tilde_t[i] = q_hat_t[i] # Predicted (LSTM)

            q_tilde[s, t] = q_tilde_t

            # Update history buffer (Autoregression)
            q_history_buffer[t] = q_tilde_t

            # 4. Hydraulic Simulation (EPANET Step)
            # SIMULATION PROXY: The pressure is not computed here. Requires WNTR/EPYNET integration.
            # p_tilde_t = EPANET(q_tilde_t, G) # <--- PLACEHOLDER FOR EPANET CALL

    return q_tilde, q_hat, U_scores

# %%
# --- Adaptive and Baseline Execution ---

q_tilde_adaptive, q_hat_adaptive, U_scores_adaptive = adaptive_sampling_simulation(
    q_test,
    p_test,
    node_models,
    node_quantiles,
    BUDGET_B,
    LOOKBACK_WINDOW
)

# Prepare slices for evaluation (ignoring lookback window steps)
q_test_slice = q_test[:, LOOKBACK_WINDOW:, :]
q_tilde_adaptive_slice = q_tilde_adaptive[:, LOOKBACK_WINDOW:, :]
q_hat_slice = q_hat_adaptive[:, LOOKBACK_WINDOW:, :]

# --- Baseline: Uniform Random Sampling ---
T, N = q_test_slice.shape[1], q_test_slice.shape[2]
q_tilde_uniform = np.zeros_like(q_test_slice)

for s in range(q_test_slice.shape[0]):
    for t in range(T):
        sampled_indices = np.random.choice(N, BUDGET_B, replace=False)
        for i in range(N):
            if i in sampled_indices:
                q_tilde_uniform[s, t, i] = q_test_slice[s, t, i] # Measured
            else:
                q_tilde_uniform[s, t, i] = q_hat_slice[s, t, i] # Predicted

print("All simulations complete.")

---

## 5. Evaluation Metrics

**Goal:** Calculate Demand RMSE (the primary metric for $\tilde{q}$) and Empirical Coverage (to validate CP), comparing the Adaptive Method with Uniform Random Sampling.

---

In [ ]:
# Flatten data for metric calculation
q_true_flat = q_test_slice.flatten()
q_tilde_adaptive_flat = q_tilde_adaptive_slice.flatten()
q_tilde_uniform_flat = q_tilde_uniform.flatten()

# 1. Demand RMSE (Eq. 9)
def calculate_rmse(q_true, q_tilde):
    return np.sqrt(np.mean((q_true - q_tilde)**2))

rmse_q_adaptive = calculate_rmse(q_true_flat, q_tilde_adaptive_flat)
rmse_q_uniform = calculate_rmse(q_true_flat, q_tilde_uniform_flat)

# 2. Coverage (Empirical) (Eq. 11)
def calculate_coverage(q_true_slice, q_hat_slice, quantiles, N_nodes):
    """Calculates the empirical coverage based on CP intervals."""
    coverage_count = 0
    total_samples = q_true_slice.size

    q_true_2d = q_true_slice.reshape(-1, N_nodes)
    q_hat_2d = q_hat_slice.reshape(-1, N_nodes)

    for i in range(N_nodes):
        Q_i = quantiles[i]
        q_true_i = q_true_2d[:, i]
        q_hat_i = q_hat_2d[:, i]

        lower_bound = q_hat_i - Q_i
        upper_bound = q_hat_i + Q_i

        is_covered = (q_true_i >= lower_bound) & (q_true_i <= upper_bound)
        coverage_count += np.sum(is_covered)

    empirical_coverage = coverage_count / total_samples
    return empirical_coverage * 100

coverage_adaptive = calculate_coverage(q_test_slice, q_hat_slice, node_quantiles, N_NODES)
target_coverage = (1 - ALPHA) * 100

### 5.2 Results Summary

Comparing the Adaptive (CP-Guided) Method against the Uniform Random Sampling baseline.

In [ ]:
print("## 📊 Evaluation Results (Synthetic Data Proxy)")
print(f"Network: {NETWORK_NAME}, Sampling Budget: {BUDGET_PERCENT*100:.0f}% (B={BUDGET_B} sensors)")

print("\n--- Demand Estimation RMSE (L/s) ---")
print(f"Ours (Adaptive): {rmse_q_adaptive:.4f}")
print(f"Uniform Random: {rmse_q_uniform:.4f}")
improvement = (rmse_q_uniform - rmse_q_adaptive) / rmse_q_uniform * 100
print(f"Improvement over Uniform: {improvement:.2f}%")

print("\n--- Conformal Prediction Coverage (%) ---")
print(f"Target Coverage (1-alpha): {target_coverage:.1f}%")
print(f"Ours (Adaptive) Empirical Coverage: {coverage_adaptive:.2f}%")

print("\n--- Next Steps for Full Replication ---")
print("To calculate Pressure RMSE and Violation Rate (Section 4.2), the hydraulic solver (EPANET) must be integrated at the placeholder in Section 4.")

# In your local terminal or a specialized cloud environment:
pip install wntr pandas numpy

In [ ]:
# Assuming you are testing the Hanoi network:
INP_FILE = 'Hanoi.inp'

# If running in Colab, you would need to mount Drive or upload the file:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
import wntr
import pandas as pd

def run_single_step_epanet(inp_file_path, demand_vector_t, node_names):
    """
    Runs an EPANET simulation for a single timestep using a custom demand vector.

    Args:
        inp_file_path (str): Path to the EPANET .inp file.
        demand_vector_t (np.array): Hybrid demand values [q_tilde_t(1), ..., q_tilde_t(N)].
        node_names (list): List of junction IDs corresponding to the demand_vector.

    Returns:
        dict: Estimated pressure values {node_id: p_tilde}
    """

    # 1. Load the network model
    wn = wntr.network.WaterDistributionModel(inp_file_path)

    # 2. Modify demands for the single step

    # Get all junctions that consume water (ignoring reservoirs/tanks)
    junctions = [name for name, element in wn.nodes.items() if isinstance(element, wntr.network.Junction)]

    # Check if the input node names match the junction names
    # This mapping is critical for WDN datasets!
    if set(junctions) != set(node_names):
        print("WARNING: Junctions in INP file do not match demand vector nodes.")
        # Attempt to map based on list order

    # Reset all base demands and set a generic pattern (constant multiplier of 1.0)
    # EPANET requires a pattern even for base demands
    pattern_name = 'CustomPattern'
    if pattern_name not in wn.patterns:
        wn.add_pattern(pattern_name, [1.0])

    for idx, node_id in enumerate(node_names):
        if node_id in wn.junctions:
            junction = wn.get_node(node_id)
            junction.base_demand = demand_vector_t[idx]
            junction.demand_pattern = pattern_name

    # 3. Configure and run a single-step simulation
    wn.options.time.duration = 3600  # 1 hour simulation
    wn.options.time.hydraulic_timestep = 3600 # Single step

    sim = wntr.sim.EpanetSimulator(wn)
    results = sim.run_sim()

    # 4. Extract results (pressure in meters)
    # The pressure data is usually stored by node ID
    pressure_series = results.node['pressure'].iloc[-1].to_dict() # Get the last timestep (t=1h)

    return {node_id: pressure_series.get(node_id, np.nan) for node_id in node_names}

In [ ]:
            # ... [Steps 1, 2, 3 complete: q_tilde_t calculated] ...

            q_tilde[s, t] = q_tilde_t
            q_history_buffer[t] = q_tilde_t

            # --- 4. Hydraulic Simulation (Algorithm 1, Line 10) ---

            # **REPLACING THE PROXY:**

            # For this step, you need the names of all nodes (e.g., J1, J2, ...).
            # Assuming you loaded them during data prep:
            # node_names = [...]

            # # --- REAL HYDRAULIC CALL (UNCOMMENT IF WNTR IS AVAILABLE) ---
            # try:
            #     # This function needs to be imported or available in the environment:
            #     p_tilde_dict = run_single_step_epanet(INP_FILE, q_tilde_t, node_names)
            #
            #     # Convert dictionary results back to numpy array p_tilde_t
            #     # This requires careful matching of node_names to dictionary keys.
            #     p_tilde_t = np.array([p_tilde_dict[name] for name in node_names])
            #
            # except Exception as e:
            #     print(f"Error running WNTR at step {t}: {e}")
            #     p_tilde_t = p_scenario[t] # Fallback to proxy

            # --- PROXY FOR DEMO (KEEP FOR INITIAL RUNS) ---
            p_tilde_t = p_scenario[t] # Simplified: p_tilde = p_true

            # STORE p_tilde_t in a new array for later pressure RMSE calculation
            # You will need to define p_tilde_adaptive = np.zeros_like(p_test)
            # and fill it here.
            # p_tilde_adaptive[s, t] = p_tilde_t